# Pulse Ox
Aubrey Stevens

In [1]:
import asyncio
import re
import sys
import time

import numpy as np
import pandas as pd

import scipy as sp
import scipy.signal


import serial
import serial.tools.list_ports


import bokeh.plotting
import bokeh.io
import bokeh.layouts
import bokeh.driving
bokeh.io.output_notebook()

notebook_url = "localhost:8888"

Loading BokehJS ...

# Section 1: Overview:

In my design of the pulse ox, I had several main objectives. I primarily focused on producing clear, accurate data and creating an interactive, user friendly interface. In order to calculate both the pulse rate and SpO2, the AC and DC data from both the IR and Red wavelengths needed to be collected. Thus, I would need to read in four separate signals with clearly distinguishable peaks. To produce distinguishable peaks, values from the sensor probe needed to be read out consistently, with little noise, and a strong signal. Additionally, the signal needed to be an entirely positive voltage for Arduino to properly collect it.

For the user interface, I wanted to create a plot that, in real-time, streamed the pulse of the user. I also wanted it to display the current pulse rate and SpO2 information, updating them as the pulse changed. However, I also wanted to ensure the pulse rate and SpO2 were only changing periodically to ensure the user could easily read the data. I wanted to build in buttons to stream the pulse rate, reset the values, save the data, and shutdown the app, as these all seemed like widgets that would be useful to a user. Additionally, I wanted to create plots to show the updating pulse rates and SpO2 data and show how they correspond to the peaks they're calculated from. Overall, I wanted to produce an accurate app that displays lots of data.

# **Section 2: Design and Design Considerations:**

## **Circuit**:
As mentioned in the overview, one of my primary goals was to create clear, accurate data. In this case, clear and accurate data are consistent, easily distinguishable peaks with little noise, corresponding closely with the readings of the sensor probe.

**Sensor Probe:**
\
To set up my sensor probe, I generally followed the set up on the pulse ox page of the BE189 website. However, I did make some changes and will go through all of the pins as listed and detailed below. 
- pin 2: Connected to digital pin 3, so it could be turned on and off from 5V to 0V, opposite of pin 3
- pin 3: Connected to a 220 $\Omega$ resistor, which is connected to digial pin 2, so it can be turned on and off from 5V to 0V, opposite of pin 2
- pin 5: Photodiode anode, connected to ground
- pin 7: Shield, connected to ground
- pin 9: Photodiode cathode, onnected to 5V through a 330 k $\Omega$ resistor


*Pins 2 and 3:*
\
Since only either the IR or red light can be emitted and detected at once, this meant that I needed to alternate which light was emitting from the LEDs. Thus, I connected pins 2 and 3 of the sensor probe to digital pins, so I could vary the voltages to alternate which light is on. We were told that the when the LEDs receive +5V, the red light is on, while when the LEDs receive -5V, the IR light is on. Whether the voltage is positive or negative is simply a denotation of which direction the current is flowing in. Thus, if we change the direction of the current, we change the sign of the voltage. Together, sensor pins 2 and 3 complete a circuit, as given in the pulse ox background. Since they complete a circuit, I only need to include 1 220 $\Omega$ resistor, as it will affect both pins 2 and 3 regardless of which pin it is connected to. As mentioned on the course website, the Red wavelength is active when +5V is flowing through the circuit, which is defined as when pin 3 of the sensor probe is on (at +5V) and pin 2 is grounded (at 0V). IR is active when -5V is flowing through the circuit, which is defined as when pin 3 of the sensor probe is grounded and pin 2 is at +5V. Thus, alternating between these voltages for the two pins allows me to switch between turning on the IR and Red wavelength light.

*Pins 5, 7, and 9:*
\
I set up pins 5, 7, and 9 as suggested in the pulse ox background. Pin 7 is a shield, which needs to be grounded, while pins 5 and 9 make up the circuit for the photodiode. While we're varying which pins are power and ground for pins 2 and 3, we want the photodiode anode to be on all the time, as it is what senses both the IR and red wavelengths. Thus, regardless of which light is on, we want the photodiode to remain in the same on state. Pin 5 is the photodiode anode, which I connected to ground as shown. Pin 9 is the photodiode cathode, which I connected to 5V through a 330 k$\Omega$ resistor. Additionally, this is where I read out the raw voltage from the sensor, as the photodiode has varying resistance based on the light it senses. Using $V = IR$ for a circuit in series, as the photodiode circuit, where the total voltage is described by $V_t = IR_t$, the voltage over the 330 k $\Omega$ resistor is described by $V_1 = IR_1$, and the voltage over the photodiode is described by $V_2=IR_2$, we get $(V_1 + V_2) = I (R_1 + R_2)$ or $V_1 = I(R_1+R_2)-V_2$. Thus, any change to the resistance of the photodiode would result in a change to the voltage over the first resistor, which we can measure.

**Sample-and-Hold Circuits:**
\
To ensure that I was accurately collecting data from the sensor probe, I decided to create a sample-and-hold circuit for both the IR and Red wavelengths. These had to be created separately, as the IR and Red data can only be collected at times when the other is not being collected. Thus, I need the IR sample-and-hold circuit to be inactive and when the red sample-and-hold circuit is active and vice versa. This allows me to collect the IR and red wavelength data separately from one another.

*Logic Pin:*
\
In order to switch my sample-and-hold circuits on and off, I needed to be able to vary my logic pin between the highest possible voltage read out from the photodiode cathode (5V since I input 5V) and the lowest possible read out (0V since I had nothing to invert the direction of the current). The switch is closed (on) when the logic pin has a higher voltage than the input, meaning that I want the logic pin to be 5V when I want the sample-and-hold to be on and 0V when off. Thus, I connected them to digital pins 2 and 3 of my Arduino, the same digital pins I was using to alternate which light source was on. 

*Logic Reference:*
\
Since my logic pin is switching between 0 and 5V, I needed to ensure my logic reference was somewhere between 0 and 5V. Thus, I decided to set my logic reference to 2.5V. I did this by creating a voltage divider with two 220 $\Omega$ resistors. 

*Capacitor:*
\
I decided to use a 10 ms sample delay, and experimented with a few different capacitances. 0.01 uF or 10 nF appeared to be the best capacitance to use for the circuit.

*$+V_{in}$ and $-V_{in}$*
\
I initially used a 5V powerBRICK to supply the $+V_{in}$ and $-V_{in}$, but it didn't appear to be a high enough voltage to supply the sample-and-hold circuits or the filters later on. Therefore, I swapped it out for a 12V powerBRICK, which produced nicer data from the sample-and-hold circuits.

**Low-Pass Filter:**
\
Both the raw data from the sensor and the data after the sample-and-hold circuits were quite noisy and needed considerable filtering. Building a low-pass filter would allow me to filter out a lot of the high frequency noise in the data. I decided to filter out all frequencies above 5 Hz or about 300 bpm, which is a much higher heart rate than any human should have. I chose to actively filter using a Sallen-Key low-pass filter, as it would remove more noise than a passive filter. To calculate the values for the capacitors and resistors I would use, I used the approximation:

$f_{cut} = \frac{1}{2\pi RC}$, where $f_{cut}$ is the cutoff frequency of 5 Hz
\
Solving for R, we get:
$R = \frac{1}{2\pi C f_{cut}}$
$R = \frac{1}{10\pi C}$

For simplicity, I set R1=R2 and C1=C2. Since the most we can solve for is a relationship between R and C, I simply plugged in different capacitances we had in the lab until I got a convenient resistance value (again, ones that were in the lab). I initially used 1 uF capacitors and 33 k $\Omega$ resistors for both the IR and Red low-pass filters, but it let too much noise through in the IR data. Thus, for the IR data, I ended up using a 0.1 uF capacitor and a 330 k$\Omega$ resistor. This appeared to filter out much of the undesired noise. The Red wavelength data seemed fine with the 0.1 uF capacitor, so I kept it.

I set up my Sallen-Key low-pass filter the same way as shown in lesson 26. I designated Z1 and Z2 as the resistors and Z3 and Z4 as the capacitors (see diagram below)

![](sallen_key.png)

*DC Data:*
\
After the low-pass-filter, I collected the data from both the IR and Red wavelengths in order to compute the SpO2. Upon high-pass filtering, the data will lose its DC information, as DC is at a much lower frequency than AC. Thus, all of the DC data will be filtered out, leaving only the AC.


**High-Pass Filter:**
\
As mentioned above, the high-pass filter filters out the DC frequencies, only leaving the AC signal. I set up my high-pass filter almost exactly the same as the low-pass filter but used a different cutoff frequency and swapped the positions of the resistors and capacitors.

I used 0.5 Hz as my cutoff frequency, as it corresponds to about 30 bpm, which is a very low heartrate. Following the methods described in the low-pass filter, I calculated a capacitance of 1 uF and a resistance of 330 k $\Omega$. This appeared to filter out my low frequency noise quite well. This time, I designated Z1 and Z2 as capacitors and Z3 and Z4 as resistors in the diagram above. After this filtering, my data was very faint and needed to be amplified.


**Amplification:**
\
After the high-pass filtering, my data was quite faint and needed to be amplified. I followed lesson 22 to create a non-inverting amplifier to amplify my signal. I initially wasn't sure how much I would need to amplify my signal and played around with the values of my resistors until my data seemed to be amplified to a good level. When trying to find a good level of amplification, I was looking for clear, distinct peaks that took up about half of the range from 0 to 5V. I figured this would be a good amplitude for my peaks, as they were clear and easily distinguishable without any concern of going out of range. I ended up using resistor values of 15 $\Omega$ and 220 $\Omega$ for the IR pathway and 4.7 $\Omega$ and 220 $\Omega$ for the red pathway.


**Summing Amplifier:**
\
After amplification, even though I had very clear peaks, I noticed that valleys of some of my peaks were flat on the bottom, meaning that they were going into negative voltages, which couldn't be read into the Arduino. To correct for this, I decided to create a summing amplifier, as shown in lesson 23. I wasn't sure how much I needed to weight my inputs or by how much I needed to alter my data, so I played with the values of the resistors again.

In my summing amplifier, I fed my data from the amplifier through a resistor into the input of the summing amplifier along with a voltage of 5V through another resistor. I used 5 V since 5V is the maximum value the Arduino can read in. I swapped out the resistor for the 5V and adjusted the resistor for the data from the amplifier until none of my data was going below 0. For the IR pathway, I ended up using resistor values of 1.2 k $\Omega$ for the voltage from the amplifier, 7.5 k $\Omega$ for the 5V, and two 1 k $\Omega$ resistors for R3 and R4. For the red pathway, I used 10 k $\Omega$ resistors for R1 (from the amplifier), R3, and R4 and a 110 $\Omega$ resistor for the 5 V path.


**Schematics:**

<pre>
<img src="Pulse_ox_IR.jpg" alt="IR" width=450 />  <img src="Pulse_ox_red.jpg" alt="red" width=450 />

**Breadboard**
<pre>
<img src="pulse_ox_bb.jpg" alt="IR" width=800 />

### **Sketch**:
I ended up writing 2 sketches in total. The first sketch had no communication with Python and was a test sketch so that I could ensure different parts of my circuit were working. It also ended up being useful for troubleshooting later on when parts of the circuit would randomly stop working. The second sketch was my final sketch that communicated with Python to send data. For both sketches, I tried to make things as simple as possible to prevent any bugs. The final sketch is in part based on the first one. The final sketch is essentially a more complex version of my test sketch, so I'll only be describing the final one.

*Initializing:*
\
To begin, I first intitialized my variables. This primarily included the digital and analog pins I used, as well as voltage variables. Initializing the voltages as global variables is important, because not all of them are updated upon each iteration, so I need to ensure they are accessible to be printed, even if they haven't been updated. I also initialized my sample delay of 10 ms. I also initialized a variable called `light_count`, which keeps track of the number of times `printVoltage()` has looped.

*Setup():*
\
In the `Setup()` function, I began the Serial connection using `Serial.begin()` and setting the baud rate to 115200. Then, I used `pinMode()` to declare my digital pins as outputs and my analog pins as inputs. I used `digitalWrite()` to set my digital pins to low (off) to start, meaning that my pulse ox would start as off. 

*printVoltage():*
\
In the `printVoltage()` function, I begin by collecting the current time using `millis()`, which will be returned at the end of the function. I used the `light_count` variable (how many times `printVoltage()` has looped) to keep track of whether I'm going to turn on the IR or red LED and collect data from it. Since each of them takes 10 ms, this ensures that only one of them is on at a time. Thus, when `light_count` is even, I turn on the Red wavelength pathway and when it is odd, I turn on the IR pathway. Then, I collect the voltages for both AC and DC for either IR or Red using 1 `analogRead()` and I write out the values of the digital pins (high or low) using `digtialWrite()`. Lastly, I print out all of the AC and DC voltages to the Serial moniter for both IR and red, regardless of if they've been updated, as the last value they're stored as is the last update. This allows the values to be read into Python for use in the Bokeh app. The function returns the time the data was acquired to be used to determine when to collect the next data point.


*loop()*
\
For the `loop()` function, I primarily followed the sketch from lesson 16 to communicate with Python. This allows me to stream in and request voltages through Python, giving me much greater functionality with my Bokeh app. Within it, I call `printVoltage()` collect data when desired through Python. Within the function, I'm still acquiring data and calling `printVoltage()` every 10 ms, it's just not reading into Python unless requested.

```cpp
const int LPreadIRPin1 = A1;
const int LPreadRedPin1 = A2;
const int HPreadIRPin2 = A3;
const int HPreadRedPin2 = A4;
const int outRedPin1 = 2;
const int outIRPin1 = 3;
const unsigned long sampleDelay = 10;
unsigned long lastSampleTime = 0;

int lightCount = 0;
int voltageIR1 = 0;
int voltageIR2 = 0;
int voltageRed1 = 0;
int voltageRed2 = 0;

const int HANDSHAKE = 0;
const int VOLTAGE_REQUEST = 1;
const int ON_REQUEST = 2;
const int STREAM = 3;
const int READ_DAQ_DELAY = 4;

// Initially, only send data upon request
int daqMode = ON_REQUEST;

// Default time between data acquisition is 100 ms
int daqDelay = 10;

// String to store input of DAQ delay
String daqDelayStr;


// Keep track of last data acquistion for delays
unsigned long timeOfLastDAQ = 0;


unsigned long printVoltage() {
  // Read value from analog pin
  unsigned long timeMilliseconds = millis();
    if (lightCount % 2 == 0) {
      voltageRed1 = analogRead(LPreadRedPin1);
      voltageRed2 = analogRead(HPreadRedPin2);
      digitalWrite(outRedPin1, HIGH);
      digitalWrite(outIRPin1, LOW);
    }
    else {
      voltageIR1 = analogRead(LPreadIRPin1);
      voltageIR2 = analogRead(HPreadIRPin2);
      digitalWrite(outRedPin1, LOW);
      digitalWrite(outIRPin1, HIGH);
    }
    lightCount += 1;
    Serial.print(String(timeMilliseconds, DEC));
    Serial.print(",");
    Serial.print(String(voltageIR1));
    Serial.print(",");
    Serial.print(String(voltageIR2));
    Serial.print(",");
    Serial.print(String(voltageRed1));
    Serial.print(",");
    Serial.println(String(voltageRed2));

  // Return time of acquisition
  return timeMilliseconds;
}


void setup() {
  // Initialize serial communication
  Serial.begin(115200);
  pinMode(outRedPin1, OUTPUT);
  pinMode(outIRPin1, OUTPUT);
  pinMode(LPreadIRPin1, INPUT);
  pinMode(LPreadRedPin1, INPUT);
  
  digitalWrite(outIRPin1, LOW);
  digitalWrite(outRedPin1, LOW);
}


void loop() {
  // If we're streaming
  if (daqMode == STREAM) {
    if (millis() - timeOfLastDAQ >= daqDelay) {
      timeOfLastDAQ = printVoltage();
      
    }
  }

  // Check if data has been sent to Arduino and respond accordingly
  if (Serial.available() > 0) {
    // Read in request
    int inByte = Serial.read();

    // If data is requested, fetch it and write it, or handshake
    switch(inByte) {
      case VOLTAGE_REQUEST:
        timeOfLastDAQ = printVoltage();
        break;
      case ON_REQUEST:
        daqMode = ON_REQUEST;
        break;
      case STREAM:
        daqMode = STREAM;
        break;
      case READ_DAQ_DELAY:
        // Read in delay, knowing it is appended with an x
        daqDelayStr = Serial.readStringUntil('x');

        // Convert to int and store
        daqDelay = daqDelayStr.toInt();

        break;
      case HANDSHAKE:
        if (Serial.availableForWrite()) {
          Serial.println("Message received.");
        }
        break;
    }
  }
}
```

### Python Code:

The general code I used was a modified version of the code in lesson 16. I chose this method of streaming and displaying my data, as it would allow me to display real time updates in the detected heart rate. It also allowed me to build a user interface with various widgets for the user to stream data, clear the data, save the data, and shut down the app. However, I did end up making quite a few modifications to the code. These modifications allowed me to read in all four of the voltages, calculate the heartrate and SpO2 in real time, and display them.

**Streaming Data:**
\
To stream in the data, I used the code from lesson 16 with some modifications. From the lesson, the provided code sets up a serial connection between Python and Arduino. This allows Python to read out data from Arduino. This data is then streamed in upon request to Python and plotted.

Since I needed the AC and DC data from both the IR and Red wavelengths to calculate the SpO2, I needed to read in the four voltages. I modified the code to read in these four voltages. Since IR the IR data corresponds most accurately to the actual pulse rate, I chose to only stream the IR AC voltage on the plot. The AC voltage had been fully filtered and amplified, making it the most accurate representation of the pulse. Since the DC data was collected for the SpO2 calculation, I chose not to display it, as the pulses weren't very apparent in the data. Since the IR and Red data alternate, and the IR data is what is being used to calculate the pulse rate, I figured it would be confusing to display both. Thus, I chose to only display the data the pulse was being computed based on, the IR AC data.


**Pulse Rate**
\
I wrote a function, `pulse_rate()` to compute the pulse rate from the IR AC data. It takes an input of the data being used to compute the pulse rate, and `time_bet_peaks` in as arguments. `time_bet_peaks` is the minimum length of time in seconds that is allowed between peaks in the data, which was created upon having an issue with computing too fast of a pulse. 

There are two peaks in each pulsation. This is a result of the opening and closing of different valves in the heart to pump blood. This results in variations in the movement of blood in the body. There are two cycles of valves opening and closing for each pump, resulting in the first large peak, followed by a second, smaller peak. This can be seen in the IR AC data. However, this presents a problem in finding the peaks of the data, as `scipy.signal.find_peaks()` was detecing both the first and second peak of the data. However, upon talking to Heather, she suggested using the parameter, `width` to ensure only peaks a certain distance from each other were collected. Since the first larger peak and the second smaller peak happen in relatively quick succession, while distinct pulses usually have some separation in time, if I ensured the minimum width was less than the possible time between two pulses but greater than the time between the between the first large peak and second smaller peak in each pulse. 

I wanted to make this an input that could be changed based on the dataset. Thus, I made it an argument in the function, `time_bet_peaks`. Then, I convert this variable into a usable width within the `scipy.signal.find_peaks()`, as the width it uses is in terms of the IR AC dataset and is therefore the indices of the datapoints. Since I used a 10 ms delay, these datapoints were each collected 10 ms apart from one another. Thus, I convert the width from seconds into how many datapoints apart the peaks are. First, I convert `time_bet_peaks` into milliseconds by multiplying it by 1000. Then, I divide it by 10, since each data point was taken 10 ms apart. This gives the minimum width in terms of the number of datapoints.

Then, I created an array of the times that correspond to each datapoint from the same data dictionary. `scipy.signal.find_peaks()` returns an index of where the peaks were found, which I could use to find the corresponding times. I used `np.take()` to extract the times corresponding to the peaks and used `np.diff()` to compute the time between peaks. Then, I took the mean of these values to find the average time between peaks and divided this by 1000, to convert from milliseconds to seconds. the current number I had was in seconds per peak (or seconds per beat), so I needed to take 1 / this value to convert to beats per second. Then, I multiplied this number by 60 to get beats per minute.

**SpO2**
\
To compute the SpO2, I wrote two functions, `mod_ratio()` and `SpO2()`. The first function computes the modulation ratio, R using the approximation given in the pulse ox background, $R = \frac{AC_1/DC_1}{AC_2/DC_2}$, where the subscript 1 refers to the red wavelength data and subscript 2 refers to the IR wavelength data. 

`mod_ratio()` takes in inputs of a dictionary `data`, a numpy array of `peaks_940`, which are the indices at which the peaks of the IR AC data are found, which were calculated in `pulse_rate()`, and the `time_bet_peaks`, which is the same variable that is described in the pulse rate function. I used these peaks for the AC data in my calculation for the IR portion of the modulation ratio. Then, I also found the peaks of the red AC data. Since the IR and red data were collected at different times from one another, I wanted to independently find the peaks of the red data. Then, I took the values from the peak indices using `np.take()`. Since the IR and red data could have different numbers of peaks, I independently calculated the numerator and denominator of the modulation ratio and took the mean of the data both the numerator and denominator before returning the final modulation ratio.

`SpO2()` takes in the arguments of the data in the form of a dictionary, along with a numpy array of the IR AC peaks and the `time_bet_peaks` that will be fed into the `mod_ratiio()` function. Then, I called the `mod_ratio()` function to determine the modulation ratio. I then defined the extinction coefficents (data from https://omlc.org/spectra/hemoglobin/summary.html) and computed the SpO2 using the equation in the pulse ox summary. I added 0.2 to this value as a calibration constant, as my SpO2 reading was typically in the high 0.7s. I added an `if` statement to ensure the SpO2 value didn't go above 1 and had it return 1 if the value was above 1.


**Bokeh**
\
I followed lesson 16 to displayed streamed plots in Bokeh with buttons for streaming, resetting the data, saving it, and shutting it down. However, I also attempted to add two additional plots: one for displaying the points of the heart rate and the other for the SpO2. I was intending to do this as an additional verification that the pulse rate and SpO2 were being calculated correctly and that the correct peaks were being found. I was able to get two plots to show up for this and to get the x-axis to update for all three plots. However, streaming the data onto the plot proved to be quite difficult. I wrote another Jupyter Notebook where I attempted to do this but spent many hours trying to get it to work without success. I left the plots to show that I did attempt to do this. 

In [42]:
def pulse_rate(data, time_bet_peaks=0.25):
    '''Takes in data as a dictionary and an optional minimum time between peaks (in seconds). 
    Returns the heartrate and the array of peaks.'''
    time_bet_peaks = time_bet_peaks * 50
    IR_AC_arr = np.array(data["V_IR_AC"])
    peaks, properties = sp.signal.find_peaks(IR_AC_arr, width=time_bet_peaks)
    time_arr = np.array(data["t"])
    times = np.diff(np.take(time_arr, peaks))
    if times.size > 0:
        bpm = 1/(np.mean(times)/1000) * 60

        return bpm, peaks
    else:
        return float("nan"), peaks

def mod_ratio(data, peaks_940, time_bet_peaks=0.25):
    '''Takes in an input of data and an array of peaks at 940 nm. Returns the modulation ratio.'''
    time_bet_peaks = time_bet_peaks * 50
    DC_660 = np.array(data["V_Red_DC"])
    AC_660 = np.array(data["V_Red_AC"])
    DC_940 = np.array(data["V_IR_DC"])
    AC_940 = np.array(data["V_IR_AC"])
    
    peaks_660 = sp.signal.find_peaks(AC_940, width=time_bet_peaks)[0]
    
    if peaks_660.size > 0 and peaks_940.size > 0:
        DC_660_peaks = np.take(DC_660, peaks_660)
        AC_660_peaks = np.take(AC_660, peaks_660)
        DC_940_peaks = np.take(DC_940, peaks_940)
        AC_940_peaks = np.take(AC_940, peaks_940)

        num = np.mean((AC_660_peaks / DC_660_peaks))
        denom = np.mean((AC_940_peaks / DC_940_peaks))


        return num/denom
    else:
        return float("nan")

def SpO2(data, peaks, time_bet_peaks=0.25):
    '''Takes in an input of data as a dictionary and an array of peaks at 940 nm. Returns the SpO2'''
    R = mod_ratio(data, peaks, time_bet_peaks)
    e_Hb1 = 3226.56
    e_Hb2 = 693.5
    e_O2_Hb1 = 319.6
    e_O2_Hb2 = 1214
    SpO2 = (e_Hb1 - e_Hb2 * R) / (e_Hb1 - e_O2_Hb1 + (e_O2_Hb2 - e_Hb2) * R) + .25
    if SpO2 > 1:
        return 1
    else:
        return SpO2

# **Section 3: Instructions for Use:**

## Basic Use:
1. Run all cells of this Jupyter Notebook
2. Clip the pulse ox sensor on a finger of choice, ensuring your finger is fully in
3. Press the stream button on the bottom left of the Bokeh app
4. Wait about 15-20 seconds to ensure a stable reading
5. Your heartrate and SpO2 will apppear on the upper righthand corner of the app
6. Click the stream button again to stop streaming
7. Press the reset button if you would like to clear the data and take your pulse again


## Other Features:
**Save your data:** Click the textbox in the upper right corner to name your file and click the save button. 
- It will save to your local directory unless specified
- .csv file will include the saved traces of the AC and DC data for both IR and Red wavelengths, times collected, the different heartrates computed, and the different SpO2s calculated.

**Shut Down:** If there are any errors or warnings, you can press this button on the bottom right corner to restart the kernal.

# **Python Code**
### Setting up Serial connection with Arduino

Functions copied from lesson 16. These functions were not altered

In [23]:
def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port


def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=0
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout


def read_all(ser, read_buffer=b"", **args):
    """Read all available bytes from the serial port
    and append to the read buffer.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. `**args` appears, but is never used. This is for
       compatibility with `read_all_newlines()` as a
       drop-in replacement for this function.
    """
    # Set timeout to None to make sure we read all bytes
    previous_timeout = ser.timeout
    ser.timeout = None

    in_waiting = ser.in_waiting
    read = ser.read(size=in_waiting)

    # Reset to previous timeout
    ser.timeout = previous_timeout

    return read_buffer + read


def read_all_newlines(ser, read_buffer=b"", n_reads=4):
    """Read data in until encountering newlines.

    Parameters
    ----------
    ser : serial.Serial() instance
        The device we are reading from.
    n_reads : int
        The number of reads up to newlines
    read_buffer : bytes, default b''
        Previous read buffer that is appended to.

    Returns
    -------
    output : bytes
        Bytes object that contains read_buffer + read.

    Notes
    -----
    .. This is a drop-in replacement for read_all().
    """
    raw = read_buffer
    for _ in range(n_reads):
        raw += ser.read_until()

    return raw

## Functions for requesting and reading in voltages

In [24]:
def parse_read(read):
    """Parse a read with time, volage data

    Parameters
    ----------
    read : byte string
        Byte string with comma delimited time/voltage
        measurements.

    Returns
    -------
    time_ms : list of ints
        Time points in milliseconds.
    voltage : list of floats
        Voltages in volts.
    remaining_bytes : byte string
        Remaining, unparsed bytes.
    """
    time_ms = []
    V_IR_DC_lst = []
    V_IR_AC_lst = []
    V_Red_DC_lst = []
    V_Red_AC_lst = []

    # Separate independent time/voltage measurements
    pattern = re.compile(b"\d+|,")
    raw_list = [
        b"".join(pattern.findall(raw)).decode()
        for raw in read.split(b"\r\n")
    ]

    for raw in raw_list[:-1]:
        try:
            t, V_IR_DC, V_IR_AC, V_Red_DC, V_Red_AC = raw.split(",")
            time_ms.append(int(t))
            V_IR_DC_lst.append(int(V_IR_DC) * 5 / 1023)
            V_IR_AC_lst.append(int(V_IR_AC) * 5 / 1023)
            V_Red_DC_lst.append(int(V_Red_DC) * 5 / 1023)
            V_Red_AC_lst.append(int(V_Red_AC) * 5 / 1023)
        except:
            pass

    if len(raw_list) == 0:
        return time_ms, V_IR_DC_lst, V_IR_AC_lst, V_Red_DC_lst, V_Red_AC_lst, b""
    else:
        return time_ms, V_IR_DC_lst, V_IR_AC_lst, V_Red_DC_lst, V_Red_AC_lst, raw_list[-1].encode()


def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    t, V_IR_DC, V_IR_AC, V_Red_DC, V_Red_AC = raw.rstrip().split(",")
    V_IR_DC = int(V_IR_DC) * 5 / 1023
    V_IR_AC = int(V_IR_AC) * 5 / 1023
    V_Red_DC = int(V_Red_DC) * 5 / 1023
    V_Red_AC = int(V_Red_AC) * 5 / 1023
    

    return int(t), V_IR_DC, V_IR_AC, V_Red_DC, V_Red_AC


def request_single_voltage(arduino):
    """Ask Arduino for a single data point"""
    # Ask Arduino for data
    arduino.write(bytes([VOLTAGE_REQUEST]))

    # Read in the data
    raw = arduino.read_until()

    # Parse and return
    return parse_raw(raw)

In [25]:
# Set up connection
HANDSHAKE = 0
VOLTAGE_REQUEST = 1
ON_REQUEST = 2;
STREAM = 3;
READ_DAQ_DELAY = 4;

port = find_arduino()
arduino = serial.Serial(port, baudrate=115200)
handshake_arduino(arduino)

In [26]:
# Set up data dictionaries
stream_data = dict(prev_array_length=0, t=[], V_IR_DC=[], V_IR_AC=[], V_Red_DC=[], V_Red_AC=[], mode="on demand")
calc_data = dict(t=[], V_IR_DC=[], V_IR_AC=[], V_Red_DC=[], V_Red_AC=[], pulse=[0], blood_ox=[0])

In [27]:
async def daq_stream_async(
    arduino, data, delay=10, n_trash_reads=5, n_reads_per_chunk=4, reader=read_all_newlines
):
    """Obtain streaming data"""
    # Specify delay
    arduino.write(bytes([READ_DAQ_DELAY]) + (str(delay) + "x").encode())

    # Current streaming state
    stream_on = False

    # Receive data
    read_buffer = [b""]
    while True:
        if data["mode"] == "stream":
            # Turn on the stream if need be
            if not stream_on:
                arduino.write(bytes([STREAM]))

                # Read and throw out first few reads
                i = 0
                while i < n_trash_reads:
                    _ = arduino.read_until()
                    i += 1

                stream_on = True

            # Read in chunk of data
            raw = reader(
                arduino, read_buffer=read_buffer[0], n_reads=n_reads_per_chunk
            )

            # Parse it, passing if it is gibberish
            try:
                t, V_IR_DC, V_IR_AC, V_Red_DC, V_Red_AC, read_buffer[0] = parse_read(raw)

                # Update data dictionary
                data["t"] += t
                data["V_IR_DC"] += V_IR_DC
                data["V_IR_AC"] += V_IR_AC
                data["V_Red_DC"] += V_Red_DC
                data["V_Red_AC"] += V_Red_AC
            except:
                pass
        else:
            # Make sure stream is off
            stream_on = False

        # Sleep 80% of the time before we need to start reading chunks
        await asyncio.sleep(0.8 * n_reads_per_chunk * delay / 1000)


daq_task = asyncio.create_task(daq_stream_async(arduino, stream_data))

## Functions for Plotting

In [28]:
def plot(mode):
    """Build a plot of voltage vs time data"""
    # Set up plot area
    p = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="voltage (V)",
        title="AC IR Pulse Signal",
        y_range=[-0.2, 5.2],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p.border_fill_color = "whitesmoke"

    # Defined the data source
    source = bokeh.models.ColumnDataSource(data=dict(t=[], V_IR_DC=[], V_IR_AC=[], V_Red_DC=[], V_Red_AC=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    if mode == 'stream':
        p.line(source=source, x="t", y="V_IR_AC")
    else:
        p.circle(source=source, x="t", y="V_IR_AC")

    # Put a phantom circle so axis labels show before data arrive
    phantom_source = bokeh.models.ColumnDataSource(data=dict(t=[0], V_IR_DC=[0], V_IR_AC=[0], V_Red_DC=[0], V_Red_AC=[0]))
    p.circle(source=phantom_source, x="t", y="V_IR_AC", visible=False)

    return p, source, phantom_source

In [29]:
def plot_pulse(mode, x_range):
    """Build a plot of voltage vs time data"""
    # Set up plot area
    p_pulse = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="Heart Rate (bpm)",
        title="Pulse Rate (bpm)",
        x_range=x_range,
        y_range=[-0.2, 5.2],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p_pulse.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p_pulse.border_fill_color = "whitesmoke"

    # Defined the data source
    pulse_source = bokeh.models.ColumnDataSource(data=dict(t=[], pulse_rate=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    if mode == 'stream':
        p_pulse.line(source=pulse_source, x="t", y="pulse_rate")
        p_pulse.circle(source=pulse_source, x="t", y="pulse_rate")

    # Put a phantom circle so axis labels show before data arrive
    pulse_phantom_source = bokeh.models.ColumnDataSource(data=dict(t=[], pulse_rate=[]))

    return p_pulse, pulse_source, pulse_phantom_source

In [63]:
def plot_blood_ox(mode, x_range):
    """Build a plot of voltage vs time data"""
    # Set up plot area
    p_blood_ox = bokeh.plotting.figure(
        frame_width=500,
        frame_height=175,
        x_axis_label="time (s)",
        y_axis_label="Blood Oxygen Saturation",
        title="Blood Oxygen Saturation",
        x_range=x_range,
        y_range=[-0.2, 5.2],
        toolbar_location="above",
    )

    # No range padding on x: signal spans whole plot
    p_blood_ox.x_range.range_padding = 0

    # We'll sue whitesmoke backgrounds
    p_blood_ox.border_fill_color = "whitesmoke"

    # Defined the data source
    blood_ox_source = bokeh.models.ColumnDataSource(data=dict(t=[], blood_ox=[]))

    # If we are in streaming mode, use a line, dots for on-demand
    if mode == 'stream':
        p_blood_ox.circle(source=blood_ox_source, x="t", y="blood_ox")
        p_blood_ox.line(source=blood_ox_source, x="t", y="blood_ox")

    # Put a phantom circle so axis labels show before data arrive
    blood_ox_phantom_source = bokeh.models.ColumnDataSource(data=dict(t=[], blood_ox=[]))

    return p_blood_ox, blood_ox_source, blood_ox_phantom_source

In [64]:
def controls(mode):
    if mode == "stream":
        acquire = bokeh.models.Toggle(label="stream", button_type="success", width=100)
        save_notice = bokeh.models.Div(
            text="<p>No streaming data saved.</p>", width=165
        )
        pulse_rate = bokeh.models.Div(text="<p>Not enough data.</p")
        blood_ox = bokeh.models.Div(text="<p>Not enough data.</p")
    else:
        acquire = bokeh.models.Button(label="acquire", button_type="success", width=100)
        save_notice = bokeh.models.Div(
            text="<p>No on-demand data saved.</p>", width=165
        )
        pulse_rate = bokeh.models.Div(text=("Pulse Rate: {}".format(calc_data["pulse"][0])))
        blood_ox = bokeh.models.Div(text=("Blood Oxygen Saturation: {}".format(calc_data["blood_ox"][0])))

    save = bokeh.models.Button(label="save", button_type="primary", width=100)
    reset = bokeh.models.Button(label="reset", button_type="warning", width=100)
    file_input = bokeh.models.TextInput(
        title="file name", value=f"{mode}.csv", width=165
    )

    return dict(
        acquire=acquire,
        reset=reset,
        save=save,
        file_input=file_input,
        save_notice=save_notice,
        pulse_rate=pulse_rate,
        blood_ox=blood_ox,
    )

In [65]:
def layout(p_stream, p_pulse, p_blood_ox, ctrls, calc_data):
    buttons = bokeh.layouts.row(
        bokeh.models.Spacer(width=30),
        ctrls["acquire"],
        bokeh.models.Spacer(width=295),
        ctrls["reset"],
    )
    left = bokeh.layouts.column(p_stream,
                                bokeh.models.Spacer(height=15),
                                p_pulse,
                                bokeh.models.Spacer(height=15),
                                p_blood_ox,
                                buttons)
    right = bokeh.layouts.column(
        ctrls["pulse_rate"],
        ctrls["blood_ox"],
        bokeh.models.Spacer(height=25),
        ctrls["file_input"],
        ctrls["save"],
        ctrls["save_notice"],
    )
    return bokeh.layouts.row(
        left, right, spacing=30, margin=(30, 30, 30, 30), background="whitesmoke",
    )

### Code for Widgets

In [66]:
def acquire_callback(arduino, calc_data, rollover):
    # Pull t and V values from stream or request from Arduino
    if stream_data["mode"] == "stream":
        t = stream_data["t"][-1]
        V_IR_DC = stream_data["V_IR_DC"][-1]
        V_IR_AC = stream_data["V_IR_AC"][-1]
        V_Red_DC = stream_data["V_Red_DC"][-1]
        V_Red_AC = stream_data["V_Red_AC"][-1]
            
        
    else:
        t, V_IR_DC, V_IR_AC, V_Red_DC, V_Red_AC = request_single_voltage(arduino)


    # Send new data to plot
    new_data = dict(t=[t / 1000], V_IR_DC=[V_IR_DC], V_IR_AC=[V_IR_AC], V_Red_DC=[V_Red_DC], V_Red_AC=[V_Red_AC])
    source.stream(new_data, rollover=rollover)

    # Update the phantom source to keep the x_range on plot ok
    phantom_source.data = new_data

In [67]:
def stream_callback(arduino, stream_data, new):
    if new:
        stream_data["mode"] = "stream"
    else:
        stream_data["mode"] = "on-demand"
        arduino.write(bytes([ON_REQUEST]))

    arduino.reset_input_buffer()

In [68]:
def reset_callback(mode, data, source, phantom_source, controls):
    # Turn off the stream
    if mode == "stream":
        controls["acquire"].active = False

    # Black out the data dictionaries
    data["t"] = []
    data["V_IR_DC"] = []
    data["V_IR_AC"] = []
    data["V_Red_DC"] = []
    data["V_Red_AC"] = []

    # Reset the sources
    source.data = dict(t=[], V_IR_DC=[], V_IR_AC=[], V_Red_DC=[], V_Red_AC=[])
    phantom_source.data = dict(t=[0], V_IR_DC=[0], V_IR_AC=[0], V_Red_DC=[0], V_Red_AC=[0])

In [69]:
def save_callback(mode, data, controls):
    # Convert data to data frame and save
    df = pd.DataFrame(data={"time (ms)": data["t"], 
                            "voltage IR DC(V)": data["V_IR_DC"],
                            "voltage IR AC(V)": data["V_IR_AC"],
                            "voltage Red DC(V)": data["V_Red_DC"],
                            "voltage Red AC(V)": data["V_Red_AC"],
                           })
    df.to_csv(controls["file_input"].value, index=False)

    # Update notice text
    notice_text = "<p>" + ("Streaming" if mode == "stream" else "On-demand")
    notice_text += f" data was last saved to {controls['file_input'].value}.</p>"
    controls["save_notice"].text = notice_text

In [70]:
def disable_controls(controls):
    """Disable all controls."""
    for key in controls:
        controls[key].disabled = True


def shutdown_callback(
    arduino, daq_task, stream_data, stream_controls
):
    # Disable controls
    disable_controls(stream_controls)

    # Strop streaming
    stream_data["mode"] = "on-demand"
    arduino.write(bytes([ON_REQUEST]))

    # Stop DAQ async task
    daq_task.cancel()

    # Disconnect from Arduino
    arduino.close()

### Updating Data as it Streams

In [76]:
def stream_update(data, source, calc_data, phantom_source, controls, rollover):
    # Update plot by streaming in data
    new_data = {
        "t": np.array(data["t"][data["prev_array_length"] :]) / 1000,
        "V_IR_DC": data["V_IR_DC"][data["prev_array_length"] :],
        "V_IR_AC": data["V_IR_AC"][data["prev_array_length"] :],
        "V_Red_DC": data["V_Red_DC"][data["prev_array_length"] :],
        "V_Red_AC": data["V_Red_AC"][data["prev_array_length"] :],
    }
    source.stream(new_data, rollover)
    

    # Adjust new phantom data point if new data arrived
    if len(new_data["t"]) > 0:
        phantom_source.data = dict(t=[new_data["t"][-1]], 
                                   V_IR_DC=[new_data["V_IR_DC"][-1]],
                                   V_IR_AC=[new_data["V_IR_AC"][-1]],
                                   V_Red_DC=[new_data["V_Red_DC"][-1]],
                                   V_Red_AC=[new_data["V_Red_AC"][-1]],
                                  )
        
        
        data_len = len(stream_data["t"])
        if data_len > 99 and data_len%25==0:
            calc_data["t"] = stream_data["t"][-100:-1]
            calc_data["V_IR_DC"] = stream_data["V_IR_DC"][-100:-1]
            calc_data["V_IR_AC"] = stream_data["V_IR_AC"][-100:-1]
            calc_data["V_Red_DC"] = stream_data["V_Red_DC"][-100:-1]
            calc_data["V_Red_AC"] = stream_data["V_IR_AC"][-100:-1]

            pulse_rate2, peaks = pulse_rate(calc_data)
            blood_ox = SpO2(calc_data, peaks)
            calc_data["pulse"] += [pulse_rate2 for i in range(26)]
            calc_data["blood_ox"] += [blood_ox for i in range(26)]
            if (bool(np.isnan(pulse_rate2)) is False):
                controls["pulse_rate"].text = "Pulse Rate: {:d} bpm".format(int(pulse_rate2))
                controls["blood_ox"].text = "Blood Oxygen Saturation: {:.2f} %".format(blood_ox * 100)
                
        else:
            calc_data["pulse"] += [float("nan")]
            calc_data["blood_ox"] += [float("nan")]
            calc_data["t"] += [new_data["t"][-1]]
                
    data["prev_array_length"] = len(data["t"])

In [77]:
def potentiometer_app(
    arduino, stream_data, calc_data, daq_task, rollover=400, stream_plot_delay=90,
):
    def _app(doc):
        # Plots
        p_stream, stream_source, stream_phantom_source = plot("stream")
        x_range = p_stream.x_range
        p_pulse, pulse_source, pulse_phantom_source = plot_pulse("stream", x_range)
        p_blood_ox, pulse_blood_ox, blood_ox_phantom_source = plot_blood_ox("stream", x_range)

        # Controls
        stream_controls = controls("stream")

        # Shut down
        shutdown_button = bokeh.models.Button(
            label="shut down", button_type="danger", width=100
        )

        # Layouts
        stream_layout = layout(p_stream, p_pulse, p_blood_ox, stream_controls, calc_data)

        # Shut down layout
        shutdown_layout = bokeh.layouts.row(
            bokeh.models.Spacer(width=675), shutdown_button
        )

        app_layout = bokeh.layouts.column(
            stream_layout, shutdown_layout
        )

        def _acquire_callback(event=None):
            acquire_callback(
                arduino,
                stream_data,
                calc_data,
                rollover,
            )

        def _stream_callback(attr, old, new):
            stream_callback(arduino, stream_data, new)

        def _stream_reset_callback(event=None):
            reset_callback(
                "stream",
                stream_data,
                stream_source,
                stream_phantom_source,
                stream_controls,
            )


        def _stream_save_callback(event=None):
            save_callback("stream", stream_data, stream_controls)


        def _shutdown_callback(event=None):
            shutdown_callback(
                arduino, daq_task, stream_data, stream_controls
            )

        @bokeh.driving.linear()
        def _stream_update(step):
            stream_update(stream_data, stream_source, calc_data, stream_phantom_source, stream_controls, rollover)

            # Shut down server if Arduino disconnects (commented out in Jupyter notebook)
            if not arduino.is_open:
                sys.exit()

        # Link callbacks
        stream_controls["acquire"].on_change("active", _stream_callback)
        stream_controls["reset"].on_click(_stream_reset_callback)
        stream_controls["save"].on_click(_stream_save_callback)
        shutdown_button.on_click(_shutdown_callback)

        # Add the layout to the app
        doc.add_root(app_layout)

        # Add a periodic callback, monitor changes in stream data
        pc = doc.add_periodic_callback(_stream_update, stream_plot_delay)

    return _app

### Displaying Pulse Ox Information

In [78]:
bokeh.io.show(
    potentiometer_app(arduino, stream_data, calc_data, daq_task),
    notebook_url=notebook_url,
)

# Section 4: Demonstration and Assessment:

To assess the functionality of my pulse ox, there were several factors I was looking at. Firstly, I wanted to ensure that the pulse displaying in the app showed up as clear peaks with little noise. Secondly, I checked to see if the peaks occurred at a reasonable distance from each other. Thirdly, I checked to see if the heart rate and SpO2 readings seemed reasonable. I also checked to see if the heartrate matched the frequency of peaks on the plot. 

Before implementing the heartrate and SpO2 to update with the plot, I checked that they were correctly measuring their respective values. This is how I came up with the preset width argument for `scipy.find_peaks()` and the calibration constant for the SpO2. I also counted the number of peaks and calibrated the width argument based on what gave me an accurate number of peaks. I played with this a decent amount and found that for my own pulse, about 1/4 of a second (240 bpm) was a good cutoff to ensure the pulse ox wasn't picking up on the second peak.

I spent several hours trying to get the plots of the pulse rate and blood ox to update with the calculated pulse rate and SpO2 data that was being calculated in real time but ended up having multiple errors that were very difficult to figure out. The primary one was being able to update the data at the same time as the other data, while it relied on the other data to be calculated. 

However, the heartrate seems to be consistent with the rate of peaks showing up on the plot and the SpO2 appears to be consistently in the high 90%s, which is consistent with what it should be. I have a relatively fast resting heart rate due to my medications and my pulse ox consistently reads my heart rate in the low 100s, which is close to what it is.

I also tried measuring my pulse (from my neck) while I had the pulse ox on and it seemed to align decently close to the peaks showing up and the rate at which they appeared. Overall, I think that the pulse ox I built and the corresponding app are relatively accurate to my pulse and that it's relatively easy to use.

# Section 5: Analysis of Data:

The data my pulse ox collected seems to be fairly accurate and consistent with my actual pulse rate and the pulses detected on the streamed Bokeh plot. I believe that my SpO2 was reading a bit low due to a lack of calibration, which is why I added a calibration constant of 0.25 to the return value and ensured it didn't reach above 100%. In the future, this should be calibrated through a more accurate measure.

As mentioned above, I tried using the pulse ox while measuring my heart rate through my neck. I got about 96 bpm counting from my neck and the pulse ox read about 103 bpm, which while not 100% accurate, seemed relatively close to me. I've also consistently seen measurements of 99% for my SpO2 after the 0.25 calibration constant, which is consistent with where it should be.

In an earlier version of this code, I played with varying widths for the minimum width argument in `scipy.find_peaks()` until I found a relatively good value that seemed to filter out noise caused by the second peak while still counting each larger first peak as an individual peak.

Unfortunately, I do not believe that all of these adjustments work perfectly for each person, as I've had slightly less accurate readings for other people using the pulse ox. 

You can reference the plots above for the pulse readings (these are from the IR AC data). The peaks look relatively clean and are very distinguishable from one another, suggesting that the sample-and-hold circuit, filtering, and amplification were successful. The case was similar for the Red AC data.

Thus, I think the pulse ox managed to successfully read out slight differences in voltage caused by the movement of blood in the finger. As mentioned previously, the reason why there are two peaks in the data is because there are two cycles of valves opening and closing in the heart for each pump. This results in two slight differences in the movement of blood, which are picked up by the photodiode. When the heart pumps and pushes blood through the body, this causes the finger to absorb more light, resulting in a higher voltage read by the sample-and-hold circuit (as more of a voltage is present after the first resistor).This is read by the IR wavelength of light.

The red and IR light can also measure the SpO2 of the blood by measuring the absorption of light in Hb and O2Hb. Blood differs in absorption depending on how oxygenated it is. Thus, through looking at this absorption, through knowing some constants like the extinction coefficients of Hb and O2Hb at these wavelengths, we can determine how much oxygen there is.

# Section 6: Suggestions for the Next Design Phase:

As mentioned previously, I was unfortunately not able to get the data on the two plots for pulse rate and blood ox to update with data. I spent multiple hours attempting to make this work but was unable to. This would be a good addition next time, as it would allow for additional verification that the peaks of the heart rate are being properly read in. 

Additionally, for the hardware, I had an issue where my pulse ox would occasionally stop working due to a wire moving slightly out of place. Saudering the pulse ox would ensure this didn't occur and slight movements of the pulse ox didn't result in it malfunctioning.

As for real world applications, it's a bit difficult to move around an arduino, breadboard, and laptop to take someone's pulse rate and SpO2. It would be beneficial for there to be a smaller, portable version that displays all of the information directly on it.

In [22]:
arduino.close()